In [1]:
# https://sentinelsat.readthedocs.io/en/stable/api_overview.html
from sentinelsat import SentinelAPI, make_path_filter

import requests
from glob import glob
from shapely.geometry import shape, mapping

import geojson, json
from datetime import date, timedelta

import pyproj
from shapely.ops import transform
from rasterio.mask import mask

import rasterio
import rasterio.merge
from rasterio.io import MemoryFile
from rasterio.plot import show, reshape_as_image
from rasterio.warp import reproject, calculate_default_transform

import numpy as np
from numpy.typing import NDArray

from numba import njit, prange

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaPendingDeprecationWarning)

from tqdm import tqdm

import os

from PIL import Image

import tilecloud
import gdal2tiles
import shutil

In [2]:
username = ''
password = ''

with open('credentials') as file:
    string = file.read()
    username = string.split('\n')[0].split('username=')[-1]
    password = string.split('\n')[1].split('password=')[-1]

In [3]:
def get_polygon(osm_id):
    
    req = requests.get(url='https://nominatim.openstreetmap.org/lookup', params = {'osm_ids':'R{:}'.format(osm_id), 'format':'geojson','polygon_geojson':1}).json()

    if len(req['features']) == 0:
        requests.get('https://polygons.openstreetmap.fr/',
        params={'id':osm_id})
        geojson_req=requests.get('https://polygons.openstreetmap.fr/get_geojson.py',
        params={'id':osm_id})
        return shape(geojson_req.json())
    
    return shape(req['features'][0]['geometry'])

In [4]:
def look_for_polygon_fix(city_name):
    osm_relation_id = None
    bbox = None
    # Some cities needed hard coding at the time
    # to get the correct polygons
    if city_name == 'Albuquerque, New Mexico':
        osm_relation_id = 171262
    elif city_name == 'Cleveland, Ohio':
        osm_relation_id = 182130
    elif city_name == 'Columbia, Maryland':
        osm_relation_id = 133606
    elif city_name == 'Columbia, South Carolina':
        osm_relation_id = 194000
    elif city_name == 'Mesa, Arizona':
        osm_relation_id = 110815
    elif city_name == 'Pearl City, Hawaii':
        osm_relation_id = 119264
    elif city_name == 'San Bernardino, California':
        osm_relation_id = 253639

    if osm_relation_id != None:
        geo = get_polygon(osm_relation_id)
        bbox = geo.bounds

    return osm_relation_id, bbox

In [5]:
def get_polygon_osm_relation_id_and_bbox(city_name):
    
    osm_relation_id, bbox = look_for_polygon_fix(city_name)

    if bbox == None:
        reqs=requests.get('https://nominatim.openstreetmap.org/search', params={"q": city_name, "format": "geojson"},)
        for item in reqs.json()['features']:
            if item['properties']['type']=='administrative' and item['properties']['category']=='boundary':
                osm_relation_id = item['properties']['osm_id']
                bbox = item['bbox']
                break


    bbox_tuple=(bbox[0], bbox[2], bbox[3], bbox[1])

    return osm_relation_id, bbox_tuple

In [ ]:
def get_pin_location_osm_relation_id(city_name):
    
    reqs=requests.get('https://nominatim.openstreetmap.org/search', params={"q": city_name, "format": "geojson"},)
    for item in reqs.json()['features']:
        if item['properties']['type']=='administrative' and item['properties']['category']=='admin_centre':
            osm_relation_id = item['properties']['osm_id']
            break

    return osm_relation_id

In [6]:
def get_products(bbox_tuple, date, area_relation = 'Contains', limit = None):
    api = SentinelAPI(username, password)
    products=api.query('ENVELOPE{}'.format(bbox_tuple), 
        platformname='Sentinel-2',
        processinglevel="Level-2A",
        limit = limit,  
        area_relation = area_relation, 
        date=date)

    if len(products)==0:
        return None
    
    return products

In [7]:
def download_all_images(products):
    api = SentinelAPI(username, password)
    api.download_all(products, directory_path='images/src', nodefilter=make_path_filter('*SCL_20m*.jp2'))

In [8]:
def get_image_paths(products):
    api = SentinelAPI(username, password)
    paths = []
    for folder in api.to_dataframe(products)['title']:
        paths.append(glob('images/src/{:s}/**/**/**/**/*SCL_20m*.jp2'.format(folder+'.SAFE'))[0])
    return paths

In [9]:
def merge_images(image_paths):
    crs = rasterio.open(image_paths[0]).profile['crs']
    img, affine = rasterio.merge.merge(image_paths)
    memfile = MemoryFile()
    image_file = memfile.open(driver='JP2OpenJPEG', height=img.shape[1], width=img.shape[2], count=img.shape[0], crs = crs, transform=affine, dtype=img.dtype)
    image_file.write(img)
    return image_file

In [10]:
def get_transformed_geo(geo, crs):
    projection = pyproj.Transformer.from_proj(pyproj.Proj('EPSG:4326'), crs)
    flip_coords = lambda x, y: (y,x)
    if geo.geom_type != 'Polygon':
        flipped_geo = [transform(flip_coords, polygon) for polygon in geo.geoms]
    else:
        flipped_geo = [transform(flip_coords, geo)]
    return [transform(projection.transform, polygon) for polygon in flipped_geo]


In [11]:
@njit
def calc_highlight_fraction(image, to_highlight, compare_to):
    highlighted = 0
    comparison = 0
    for y in prange(image.shape[1]):
        for x in prange(image.shape[2]):
            pixel = image[0, y, x]
            if pixel in to_highlight:
                highlighted += 1
            elif pixel in compare_to:
                comparison += 1
    return highlighted/(highlighted+comparison)

In [12]:
def highlight(image, to_highlight, with_alpha = True):
    highlighted =  np.where(image == to_highlight, 255, 0)
    if with_alpha:
        return np.reshape(np.array([highlighted, highlighted]), newshape=(highlighted.shape[0]+1,highlighted.shape[1], highlighted.shape[2]))
    return highlighted
            

In [13]:
def prepare_images(download = True):
    all_products = []
    product_map = {}
    needs_attention_image = set()
    needs_attention_osm = set()
    image_paths = {}

    if os.path.exists('product_map.json'):
        image_paths = json.load(open('product_map.json'))
        return image_paths, needs_attention_image, needs_attention_osm
    
    else:
        city_data = json.load(open('city_data.json'))

        for data in tqdm(city_data, desc='Preparing products'):
            city_name = data['city']
            start_date = date.fromisoformat(data['date'])
            end_date = start_date + timedelta(days=1)
            date_span = (start_date, end_date)

            osm_id, bbox = get_polygon_osm_relation_id_and_bbox(city_name)
            if osm_id == None or bbox == None:
                needs_attention_osm.add(city_name)
            else:    
                products = get_products(bbox, date_span, area_relation = 'Contains', limit = 1)
                if products == None:
                    products = get_products(bbox, date_span, area_relation = 'Intersects')
                    if products == None:
                        needs_attention_image.add(city_name)
                    else:
                        product_map[city_name] = list(products)
                        all_products.extend(list(products))
                        image_paths[city_name] = get_image_paths(products)
                else:
                    product_map[city_name] = list(products)
                    all_products.extend(list(products))
                    image_paths[city_name] = get_image_paths(products)
    
    print('Ready: {:n}\t Attention image: {:n}\t Attention OSM: {:n}'.format(len(set(all_products)), len(needs_attention_image), len(needs_attention_osm)))
    
    with open('product_map.json', mode = 'w') as file:
        file.write(geojson.dumps(image_paths, indent=4))
    
    if download:
        download_all_images(all_products)

    return image_paths, needs_attention_image, needs_attention_osm

In [14]:
def mask_image(img, geometry, crop=True):
    with rasterio.Env():
        masked, affine = mask(img, geometry, crop=crop)
        memfile = MemoryFile()
        image_file = memfile.open(driver='JP2OpenJPEG', count=masked.shape[0], height=masked.shape[1], width=masked.shape[2], dtype=masked.dtype, crs=img.crs, transform=affine)
        image_file.write(masked)
        return image_file

In [15]:
def reproject_image(image, dst_crs='EPSG:3857'):
    with rasterio.Env():
        source = image.read()
        dst_transform, height, width = calculate_default_transform(src_crs=image.crs, dst_crs=dst_crs, height=image.height, width=image.width, left=image.bounds.left, bottom=image.bounds.bottom, right=image.bounds.right, top=image.bounds.top)
        
        memfile = MemoryFile()
        image_file = memfile.open(driver='JP2OpenJPEG', count=1, height=width, width=height, dtype=source.dtype, crs=dst_crs, transform=dst_transform)
       
        reproject(source=rasterio.band(image, 1), destination=rasterio.band(image_file, 1), src_crs=image.crs, dst_crs=dst_crs, src_transform=image.transform, dst_transform=dst_transform)

        transformer = pyproj.Transformer.from_crs(crs_from=dst_crs, crs_to='EPSG:4326')
        sn,we = transformer.transform([image_file.bounds[0],image_file.bounds[2]], [image_file.bounds[1],image_file.bounds[3]])

        return image_file, (sn, we)

In [16]:
BANDS = {
    0: 'no_data',
    1: 'saturated_or_defective',
    2: 'dark_area_pixels',
    3: 'cloud_shadows',
    4: 'vegetation',
    5: 'not_vegetated',
    6: 'water',
    7: 'unclassified',
    8: 'cloud_medium_probability',
    9: 'cloud_high_probability',
    10: 'thin_cirrus',
    11: 'snow'
}

In [17]:
def make_folder(path):
    if '/' in path:
        child = path.split('/')[-1]
        parent = path.split('/'+child)[0]
        make_folder(parent)
    if not os.path.isdir(path):
            os.mkdir(path)

In [18]:
def process_city(city_name, paths, highlight_bands = [4, 5, 6]):
    img = merge_images(paths)
    osm_id, bbox = get_polygon_osm_relation_id_and_bbox(city_name)
    geo = get_polygon(osm_id)

    path = 'export/polygons'
    make_folder(path)
    with open('{:s}/{:s}.json'.format(path, city_name), mode = 'w') as file:
        file.write(geojson.dumps(mapping(geo), indent=4))

    usable_geo = get_transformed_geo(geo, img.profile['crs'])

    masked_image = mask_image(img, usable_geo, crop=True)
    reprojected, bounds = reproject_image(masked_image)
    
    coverages = {}

    with rasterio.Env():
        path = 'images/masked'
        make_folder(path)
        with rasterio.open('{:s}/{:s}.png'.format(path, city_name), mode='w', driver='PNG', count=1, height=reprojected.shape[0], width=reprojected.shape[1], crs=reprojected.profile['crs'], transform=reprojected.transform, dtype=np.uint8, compression='lzw') as file:
                file.write(reprojected.read())
        
        for band in highlight_bands:
            other_bands = highlight_bands.copy()
            coverage_percent = calc_highlight_fraction(reprojected.read(), to_highlight=[band], compare_to=other_bands) * 100
            band_name = BANDS[band]
            coverages[band_name] = coverage_percent
            highlighted = highlight(reprojected.read(), band)

            path = 'images/processed/{:s}'.format(band_name)
            make_folder(path)
            with rasterio.open('{:s}/{:s}.png'.format(path, city_name) ,mode='w', driver='PNG', count=highlighted.shape[0], height=highlighted.shape[1], width=highlighted.shape[2], crs=reprojected.profile['crs'], transform=reprojected.transform, dtype=np.uint8, compression='lzw') as file:
                file.write(highlighted)

            path = 'export/masks/{:s}'.format(band_name)
            make_folder(path)
            Image.fromarray(np.array(reshape_as_image(highlighted), dtype=np.uint8)).save('{:s}/{:s}.png'.format(path, city_name))



    return coverages, [*bounds[1], *bounds[0]]

In [19]:
def run():
    image_paths, _needs_attention_image, _needs_attention_osm = prepare_images(download = False)

    processed = {}
    bboxes = {}
    
    bands = np.arange(1, 11+1)
    for city_name, paths in tqdm(image_paths.items(), desc='Processing'):
        coverage, bbox_poly = process_city(city_name, paths, highlight_bands=bands)
        processed[city_name] = coverage
        bboxes[city_name] = bbox_poly

    with open('export/coverage_percent_bands_{:}-{:}.json'.format(bands[0], bands[-1]), mode = 'w') as file:
        file.write(geojson.dumps(processed, indent=4))

    open('export/bbox.json', mode='w').write(json.dumps(bboxes, indent=4))

In [20]:
@njit
def stack_image_arrays(image_1 : NDArray[np.uint8], image_2 : NDArray[np.uint8]) -> NDArray[np.uint8]:
    combined_image = np.zeros(image_1.shape, dtype=image_1.dtype)
    for y in prange(combined_image.shape[0]):
        for x in range(combined_image.shape[1]):
            for band in range(combined_image.shape[2]):
                combined_image[y][x][band] = max(image_1[y][x][band], image_2[y][x][band])

    return combined_image

In [21]:
def stack_image_tiles(image_path_1 : str, image_path_2 : str) -> NDArray[np.uint8]:
    image_1 = np.array(Image.open(image_path_1))
    image_2 = np.array(Image.open(image_path_2))
    stacked = stack_image_arrays(image_1, image_2)

    return stacked

In [22]:
def copy_and_merge_tiles(tiles_path : str, export_path : str) -> None:
    if not os.path.exists(export_path):
        os.makedirs(export_path)
    
    files_in_tiles_path : list[str] = glob('*/*/*.png', root_dir=tiles_path)
    files_in_export_path : list[str] = glob('*/*/*.png', root_dir=export_path)

    for file_path in files_in_tiles_path:
        existing_path : str | None = None
        for existing_file_path in files_in_export_path:
            if file_path == existing_file_path:
                existing_path = existing_file_path
        
        if existing_path != None:
            stacked_image = Image.fromarray(stack_image_tiles(tiles_path+file_path, export_path+file_path))
            stacked_image.save(export_path+file_path)
        else:
            os.makedirs(export_path+file_path.replace(file_path.split('/')[-1].split('\\')[-1],''), exist_ok=True)
            shutil.copy2(tiles_path+file_path, export_path+file_path)

In [23]:
def make_tiles(start_zoom=0, end_zoom=14, nb_processes=6, webviewer='none') -> None:
    image_paths, _needs_attention_image, _needs_attention_osm = prepare_images(download = False)

    first = True
    for city_name, paths in tqdm(image_paths.items(), desc='Processing'):

        bands = np.arange(1,11+1)
        for band in bands:
            band_name = BANDS[band]
            path = 'images/processed/{:s}'.format(band_name)

            gdal2tiles.generate_tiles('{:s}/{:s}.png'.format(path, city_name),
                                      'export/tiles/separate/{:s}/{:s}/'.format(city_name, band_name),
                                      zoom=[start_zoom, end_zoom],
                                      resume=True,
                                      title='SaTreeLight - {:s} - {:s}'.format(city_name, band_name), 
                                      nb_processes=nb_processes,
                                      webviewer=webviewer,
                                    )

            copy_and_merge_tiles(tiles_path='export/tiles/separate/{:s}/{:s}/'.format(city_name, band_name), 
                                 export_path='export/tiles/merged/{:s}/'.format(band_name)
                                )

            if (first):
                gdal2tiles.generate_tiles('{:s}/{:s}.png'.format(path, city_name),
                                          'export/tiles/merged/{:s}/'.format(band_name),
                                          zoom=[start_zoom, end_zoom],
                                          resume=True,
                                          title='SaTreeLight - {:s}'.format(band_name),
                                          nb_processes=nb_processes,
                                          webviewer=webviewer,
                                        )
        if (first):
            first=False

In [ ]:
make_tiles()

In [25]:
# Not worth further investigation, files get large/unparseable
def polygonize_test():
    from osgeo import gdal, ogr
    raster = gdal.Open('images/processed/vegetation/Akron, Ohio.png')
    band = raster.GetRasterBand(1)

    if not os.path.exists('test/'):
        os.makedirs('test')
    out_file = 'test/akron.geojson'
    if os.path.exists(out_file):
        os.remove(out_file)
    driver = ogr.GetDriverByName('GeoJson')
    out_data_source = driver.CreateDataSource(out_file)
    out_layer = out_data_source.CreateLayer('vegetation', srs=None)

    new_field = ogr.FieldDefn('MYFLD', ogr.OFTInteger)
    out_layer.CreateField(new_field)

    gdal.Polygonize(band, None, out_layer, 0, [], callback=None)